In [11]:
import os

import pandas as pd
import numpy as np

from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

import tensorflow

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [3]:
kaggle_train_path = "../data/train"
kaggle_test_path = "../data/test"

features_path = "../features"
painters_path = "../data/artists.csv"

kaggle_painter_names = os.listdir(kaggle_train_path)
painters = pd.read_csv(painters_path)
painters = painters["name"]

# CNN Model

In [17]:
cnn_height = 255
cnn_width = 255
cnn_batch_size = 32

In [18]:
cnn_train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

cnn_train_generator = cnn_train_datagen.flow_from_directory(
    kaggle_train_path,
    target_size=(cnn_height, cnn_width),
    batch_size=cnn_batch_size,
    class_mode='categorical'
)

cnn_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

cnn_test_generator = cnn_test_datagen.flow_from_directory(
    kaggle_test_path,
    target_size=(cnn_height, cnn_width),
    batch_size=32,
    class_mode="categorical",
    shuffle=False,
)

Found 6735 images belonging to 50 classes.
Found 1711 images belonging to 50 classes.


In [19]:
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(cnn_height, cnn_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(painters), activation='softmax')  # Number of output neurons equal to the number of painters
])

In [20]:
cnn_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

cnn_model.fit(cnn_train_generator, epochs=10)

cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(cnn_test_generator)
print(f"CNN Test Accuracy: {cnn_test_accuracy}")

Epoch 1/10
211/211 [==============================] - 238s 1s/step - loss: 3.5128 - accuracy: 0.1228
Epoch 2/10
211/211 [==============================] - 237s 1s/step - loss: 3.1393 - accuracy: 0.1957
Epoch 3/10
211/211 [==============================] - 239s 1s/step - loss: 2.9217 - accuracy: 0.2364
Epoch 4/10
211/211 [==============================] - 235s 1s/step - loss: 2.7695 - accuracy: 0.2717
Epoch 5/10
211/211 [==============================] - 236s 1s/step - loss: 2.6505 - accuracy: 0.2980
Epoch 6/10
211/211 [==============================] - 236s 1s/step - loss: 2.4893 - accuracy: 0.3271
Epoch 7/10
211/211 [==============================] - 235s 1s/step - loss: 2.3830 - accuracy: 0.3614
Epoch 8/10
211/211 [==============================] - 235s 1s/step - loss: 2.2508 - accuracy: 0.3902
Epoch 9/10
211/211 [==============================] - 233s 1s/step - loss: 2.1118 - accuracy: 0.4148
Epoch 10/10
54/54 [==============================] - 25s 453ms/step - loss: 2.3542 - accura

In [21]:
cnn_model.save("../models/cnn_10_epochs_no_validation.h5")

# Optimization of CNN

In [ ]:
import itertools

In [ ]:
dropout_rates = [0.2, 0.3, 0.4]
activation_funcs = ['relu', 'sigmoid', 'tanh']
lr_epoch = [[0.001, 40], [0.01, 30], [0.1, 20]]
kernel_size = [3, 5]

In [ ]:
opt_cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(cnn_height, cnn_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(painters), activation='softmax')  # Number of output neurons equal to the number of painters
])

In [ ]:
cnn_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

cnn_model.fit(cnn_train_generator, epochs=10)

cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(cnn_test_generator)
print(f"CNN Test Accuracy: {cnn_test_accuracy}")

# Inception V4

In [22]:
inception_height = 299
inception_width = 299
input_shape = (inception_height, inception_width, 3) #because of RGB

In [23]:
inception_train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
)

inception_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

inception_train_generator = inception_train_datagen.flow_from_directory(
    kaggle_train_path,
    target_size=(inception_height, inception_width),
    batch_size=32,
    class_mode="categorical",
    subset="training",  #Use the training subset
)

inception_validation_generator = inception_train_datagen.flow_from_directory(
    kaggle_train_path,
    target_size=(inception_height, inception_width),
    batch_size=32,
    class_mode="categorical",
    subset="validation",  #Use the validation subset
)

inception_test_generator = inception_test_datagen.flow_from_directory(
    kaggle_test_path,
    target_size=(inception_height, inception_width),
    batch_size=32,
    class_mode="categorical",
    shuffle=False,
)

Found 5411 images belonging to 50 classes.
Found 1324 images belonging to 50 classes.
Found 1711 images belonging to 50 classes.


In [24]:
inception_model = InceptionV3(
    weights="imagenet",
    include_top=False,
    input_shape=input_shape,
)

for layer in inception_model.layers:
    layer.trainable = False

In [25]:
x = GlobalAveragePooling2D()(inception_model.output)

x = Dense(1024, activation="relu")(x)

x = Dense(512, activation="relu")(x)


#dropout, early stop



inception_output = Dense(len(inception_train_generator.class_indices), activation="softmax")(x)

inception = Model(inputs=inception_model.input, outputs=inception_output)

inception.compile(optimizer=Adam(), loss="categorical_crossentropy", metrics=["accuracy"])

In [26]:
train_steps_per_epoch = inception_train_generator.samples // inception_train_generator.batch_size
val_steps_per_epoch = inception_validation_generator.samples // inception_validation_generator.batch_size

inception.fit(
    inception_train_generator,
    steps_per_epoch=train_steps_per_epoch,
    validation_data=inception_validation_generator,
    validation_steps=val_steps_per_epoch,
    epochs=10,
)

# Evaluate the model on the test set
inception_test_loss, inception_test_accuracy = inception.evaluate(inception_test_generator)
print(f"Inception Test Accuracy: {inception_test_accuracy}")

Epoch 1/10
169/169 [==============================] - 275s 2s/step - loss: 2.9794 - accuracy: 0.2540 - val_loss: 2.4341 - val_accuracy: 0.3537
Epoch 2/10
169/169 [==============================] - 269s 2s/step - loss: 2.0732 - accuracy: 0.4358 - val_loss: 2.0209 - val_accuracy: 0.4566
Epoch 3/10
169/169 [==============================] - 270s 2s/step - loss: 1.7367 - accuracy: 0.5285 - val_loss: 1.9677 - val_accuracy: 0.4741
Epoch 4/10
169/169 [==============================] - 274s 2s/step - loss: 1.4739 - accuracy: 0.5841 - val_loss: 1.8796 - val_accuracy: 0.4970
Epoch 5/10
169/169 [==============================] - 270s 2s/step - loss: 1.3266 - accuracy: 0.6155 - val_loss: 1.8634 - val_accuracy: 0.5023
Epoch 6/10
169/169 [==============================] - 270s 2s/step - loss: 1.1923 - accuracy: 0.6512 - val_loss: 1.7428 - val_accuracy: 0.5419
Epoch 7/10
169/169 [==============================] - 269s 2s/step - loss: 1.0757 - accuracy: 0.6760 - val_loss: 1.8177 - val_accuracy: 0.5328

In [27]:
inception.save("../models/inceptionv3_10_epochs.h5")